# Demo: using `Agent`, `Moderator`, and `generate_slate`

This notebook is meant to serve as a more in depth README. We refactored the code, and this will explain how to work with the new `Agent`s, `Moderator`s, and `generate_slate` functions.

## Setup

In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from generative_social_choice.objects.abstract_agents import Agent, Moderator
from generative_social_choice.objects.agents import (
    CommentAgent,
    get_polis_agent,
    CMVAgent,
    VoteAgent,
    VoteCommentsAgent,
    CommentListAgent,
    CMVSetup,
)
from generative_social_choice.objects.moderators import (
    Query1Moderator,
    Query1PrimeModerator,
    JRModerator,
    CMVQuery1Moderator,
    CMVQuery1PrimeModerator,
)
from generative_social_choice.datasets.datasets import get_dataset
from generative_social_choice.utils.helper_functions import bullet_list_to_string

from generative_social_choice.objects.committee import (
    generate_ejr_slate,
    generate_jr_slate,
)

## Agents 

Each agent is associated with:
- A prompt used for approval queries 
- A model used for approval queries 
- Some data describing the agent's views (e.g. comments, votes), which are turned into a description

In the abstract class Agent, you see that the main interface for `Agent`s is that you can call `.get_approval()` and `.get_description()`. 

To view all prompt templates associated with a particular `Agent` or `Moderator`, you can do the following:

In [39]:
CommentAgent.prompt_templates

{'agree': 'Suppose I hold the following opinion: "{self_comment}" Would I agree with this: "{comment}"? Answer yes or no without explanation.',
 'agree_unknown_disagree': 'Suppose a person holds the following opinion: "{self_comment}" Would they agree with this: "{comment}"? Answer AGREE if they would agree, answer DISAGREE if they would disagree, and answer UNKNOWN if this is not clear from the provided information.',
 'agree_unknown_disagree_explain': 'Suppose a person holds the following opinion: "{self_comment}" Would they agree with this: "{comment}"?Format your response as a Python dict with the following items: 1) key: MY_EXPLANATION, with val: your explanation for what the person is saying, and what this implies about their opinion on the comment. 2)  key: MY_VERDICT, with val: AGREE if they would agree, DISAGREE if they would disagree, and UNKNOWN if this is not clear from the provided information>.',
 'agree_disagree_best_guess': 'Suppose a person holds the following opinion:

### CommentAgent example

In [3]:
# Creating the agent
agent = CommentAgent(
    id=0,
    prompt_type="agree_disagree_best_guess_less_agree",
    model="gpt-4-0314",
    comment="SEC is better than Maxwell Dworkin",
)

In [4]:
# Getting the agent description
agent.get_description()

'SEC is better than Maxwell Dworkin'

In [5]:
# Making an approval query
approval, response_obj = agent.get_approval(
    comment="Maxwell Dworkin is 100% better than SEC"
)

In [6]:
approval

'DISAGREE'

In [7]:
response_obj

{'id': 0,
 'prompt_type': 'agree_disagree_best_guess_less_agree',
 'prompt template': 'Suppose a person holds the following opinion: "SEC is better than Maxwell Dworkin" Would they agree with this: "{comment}"?Answer AGREE if they would agree and DISAGREE if they would disagree. Only put AGREE if you are reasonably confident that they would agree, as a safe default you should put DISAGREE. Your output should always be AGREE or DISAGREE and nothing else.',
 'model': 'gpt-4-0314',
 'subject': 'agree_disagree_best_guess_less_agree',
 'comment': 'Maxwell Dworkin is 100% better than SEC',
 'prompt': 'Suppose a person holds the following opinion: "SEC is better than Maxwell Dworkin" Would they agree with this: "Maxwell Dworkin is 100% better than SEC"?Answer AGREE if they would agree and DISAGREE if they would disagree. Only put AGREE if you are reasonably confident that they would agree, as a safe default you should put DISAGREE. Your output should always be AGREE or DISAGREE and nothing el

### PolisAgent example

To work with the Polis data, use the function `get_polis_agent()` (can be found in `agents.py`). 

#### Polis agent with votes and comments

In [8]:
agent2 = get_polis_agent(
    issue="15-per-hour-seattle",
    id=33,
    prompt_type="agree_disagree_best_guess_less_agree",
)

In [9]:
print(agent2.get_description())

Person who believes the following: "(1) Reactionary liberal trash. Doesn't solve the problem. It's a systemic problem and this solution is like a band aid on a broken leg.
". Additionally, this person has the following opinions on the following assertions:
	* AGREE with "At the very least, customers across the board are going to have to pay more to accommodate the new wage. Some people will be okay with that but I worry that customers will start spending less and going out less as they adjust to higher prices."
	* AGREE with "There seems to be a fallacy that instead of lining our richie-rich pockets, business owners will simply break off a little more for their employees. I’m an owner of a small business, and I barely get paid $15 an hour."
	* AGREE with "While the arguments for the raise in the minimum wage law have been trumpeted as being “anti corporate greed”, those larger companies are, ironically, the most able to pay for the increase in wages. This will hurt small businesses the

#### Polis agent with only comments

In [10]:
agent3 = get_polis_agent(
    issue="15-per-hour-seattle",
    id=33,
    voted_comment_ids=[],
    prompt_type="agree_disagree_best_guess_less_agree",
)

In [11]:
print(agent3.get_description())

Person who believes the following 1 things: (1) Reactionary liberal trash. Doesn't solve the problem. It's a systemic problem and this solution is like a band aid on a broken leg. 


#### Polis agent with only votes

In [12]:
agent4 = get_polis_agent(
    issue="15-per-hour-seattle",
    id=33,
    written_comment_ids=[],
    prompt_type="agree_disagree_best_guess_less_agree",
)

In [13]:
print(agent4.get_description())

Person with the following opinions on the following assertions:
	* AGREE with "I think there are better ways to implement this. For instance, what about subsidies"
	* AGREE with "There seems to be a fallacy that instead of lining our richie-rich pockets, business owners will simply break off a little more for their employees. I’m an owner of a small business, and I barely get paid $15 an hour."
	* AGREE with "During the campaign, I found out that 75% of minimum-wage workers are 20 or older. I think it’s the right thing to do to bring higher incomes to this group of people."
	* AGREE with "While the arguments for the raise in the minimum wage law have been trumpeted as being “anti corporate greed”, those larger companies are, ironically, the most able to pay for the increase in wages. This will hurt small businesses the most, and I don’t think the law’s provisions do enough to accommodate this."
	* AGREE with "At the very least, customers across the board are going to have to pay more t

#### Getting an approval

In [14]:
approval, response_obj = agent2.get_approval(comment="Minimum wage sounds great to me!")

In [15]:
approval

'DISAGREE'

In [16]:
response_obj

{'id': 33,
 'prompt_type': 'agree_disagree_best_guess_less_agree',
 'prompt template': 'Suppose a person has the following opinions on the following assertions:\n- AGREE with "I don\'t think it\'s a bad thing but for smaller businesses, only the super strong will survive. It will mean that only owners who are willing and able to work every day at their own businesses to reduce labor costs will survive."\n- AGREE with "While the arguments for the raise in the minimum wage law have been trumpeted as being “anti corporate greed”, those larger companies are, ironically, the most able to pay for the increase in wages. This will hurt small businesses the most, and I don’t think the law’s provisions do enough to accommodate this."\n- AGREE with "Reactionary liberal trash. Doesn\'t solve the problem. It\'s a systemic problem and this solution is like a band aid on a broken leg."\n- AGREE with "I think there are better ways to implement this. For instance, what about subsidies"\n- AGREE with "A

### Changemyview agent example

If you want to fetch CMV agents, fetch them all at once using `CMVSetup.get_agents_from_seed` (if you want to use a fewshot prompt) or `CMVSetup.get_all_agents` (for any other prompt)

In [17]:
agent5 = CMVSetup.get_agents_from_seed(seed=0, prompt_type="cmv_basic")[0]

In [18]:
agent5.get_description()

'Just going to add tha, while new technological breakthroughs would help tremendously, we have already come across the technical breakthroughs required for the transition away from fossil fuels.  \n\nSolar panels are now the cheapest form of energy humans have ever created. That combined with incredible improvements in battery technology, heat pumps, and electrolyzers means that the transition is more an issue of implementation rather than one of discovery.'

In [19]:
approval, response_obj = agent5.get_approval(
    comment="We're not going to stop climate change without more discoveries, current tech is not sufficient"
)

In [20]:
approval

'No'

In [21]:
response_obj

{'id': 9190153,
 'prompt_type': 'cmv_basic',
 'prompt template': 'Consider a group of people deliberating on a topic. In particular, they are responding the the following comment:\n\n"CMV: Global warming will not be solved by small, piecemeal, incremental changes to our way of life but rather through some big, fantastic, technological breakthrough."\n\nYour tasks is to determine whether a particular person would feel represented by a given statement. Below I will provide some example tasks, in which I will give you the following information: 1) an opinion some particular person has, and 2) a statement somebody else made. Given this information, the task is to determine whether the person would feel represented by the statement, based on the opinion they expressed. This is a yes-no question, the only two valid answers to the task are "Yes" (the person would feel represented by the statement) or "No" (the person would not feel represented by the statement). \n\n\n\nOpinion (what person b

## Moderator

Each moderator is associated with:
- A prompt used for its query (Query1, or Query1', or whatever)
- A model used for its query 

The main interface for a `Moderator` is `.make_query()` which is wrapped in something called `.query1()` if it's a Query1 agent of `.query1prime()` if it's a Query1' agent. This class is pretty flexible. Typically we envision a moderator as something which takes in a list of agents, though this isn't specified in the base class. 

### Query1Moderator

In [22]:
moderator = Query1Moderator(id=0, prompt_type="basic_explain_common_ground")

In [23]:
query2_prompt_type = "agree_disagree_best_guess_less_agree"
issue = "15-per-hour-seattle"
good_author_ids = get_dataset("polis").get_good_author_ids(issue=issue)
agents = [
    get_polis_agent(
        issue=issue, id=author_id, prompt_type=query2_prompt_type, voted_comment_ids=[]
    )
    for author_id in good_author_ids
]

Warning: `prompt_type="basic_explain_common_ground"` takes a while since it generates a lot of text (CoT). Switch to "basic" if you're just testing.

In [24]:
response, response_obj = moderator.query1(agents=agents)

In [25]:
response

'Although raising the minimum wage to $15 per hour is a step towards addressing income inequality, it may not be the most effective solution, as it could negatively impact small businesses and increase the risk of job automation.'

In [26]:
response_obj["STEP 1"]  # response_obj contains all CoT steps

'The topic being discussed is raising the minimum wage to $15 per hour. The key axes of disagreement are the impact on small businesses, job losses, the potential for automation, income inequality, and the effectiveness of addressing the problem.'

### Query1PrimeModerator

In [27]:
moderator_prime = Query1PrimeModerator(
    id=0, prompt_type="basic_more_specific"
)  # using a non CoT prompt for the demo so it doesn't take so long

In [28]:
excluded_comments = [
    "Minimum wage is a step in the right direction but we need to also address income inequality"
]

In [29]:
response, response_obj = moderator_prime.query1_prime(
    agents=agents, excluded_comments=excluded_comments
)

In [40]:
response

'While we have made progress in renewable energy and other technologies to address global warming, it is clear that we need a combination of systemic changes, government regulation, and innovative breakthroughs to make a significant impact. Our current global challenges require us to rethink our societal values and work together, prioritizing the well-being of both humans and the environment. Given the complexity and scale of the problem, there is no one-size-fits-all solution, and we must embrace a variety of approaches to mitigate the effects of climate change and ensure a sustainable future.'

### CMVQuery1Moderator 

In [41]:
moderator2 = CMVQuery1Moderator(id=0, prompt_type="cmv_basic", seed=0)

In [42]:
cmv_agents = CMVSetup.get_agents_from_seed(seed=0, prompt_type="cmv_basic")

In [43]:
response, response_obj = moderator2.query1(agents=cmv_agents)

In [44]:
response

'Addressing global warming requires a combination of existing technological breakthroughs and systemic transformations in our economic and social structures. While small incremental changes may not solve the issue entirely, they play a vital role in supporting larger innovative solutions. Cooperation, adaptability, and a shift towards sustainable practices are necessary for meaningful progress against climate change.'

### CMVQuery1PrimeModerator

In [45]:
moderator2_prime = CMVQuery1PrimeModerator(id=0, prompt_type="cmv_basic", seed=0)

In [46]:
response, response_obj = moderator2_prime.query1_prime(
    agents=cmv_agents, excluded_comments=["Global warming is a hoax"]
)

In [47]:
response

'Addressing global warming requires a comprehensive approach that includes both existing technological advancements and ongoing lifestyle changes. There may still be a need for significant breakthroughs and systemic transformations, but we cannot rely solely on these alone. Cooperation, adaptation, and political will from governments, industries, and individuals are crucial for our collective survival and well-being.'

## Generating JR/EJR slates 

### Example JR slate

In [25]:
query2_prompt_type = "agree_disagree_best_guess_less_agree"
issue = "15-per-hour-seattle"
good_author_ids = get_dataset("polis").get_good_author_ids(issue=issue)
agents = [
    get_polis_agent(
        issue=issue, id=author_id, prompt_type=query2_prompt_type, voted_comment_ids=[]
    )
    for author_id in good_author_ids
]

moderator = Query1Moderator(id=0, prompt_type="basic")

In [48]:
committee_supporters = generate_jr_slate(
    agents=agents,
    moderator=moderator,
    k=5,
    experiment_type="polis",
    log_dir_name="demos/",
)

Running JR...
Step 1: (current committee {})
Query 1 response: Raising the minimum wage is a crucial step in addressing income inequality and ensuring employees receive a livable wage, but the increase should be gradual and consider the potential impacts on small businesses and the adoption of automation in industries with unskilled workers.
Testing agent 215 (comment=Person who believes the following 1 things: (1) Businesses, small or large, shouldn't be able to pay workers anything below a livable wage. Why should we subsidize businesses that depend on inadequate wages? If a business's product is good, then this shouldn't be a challenge. )
Response: AGREE
Removing agent 215
Testing agent 85 (comment=Person who believes the following 1 things: (1) It's just going to speed up the adoption of robotics in industries with unskilled or low-skilled workers. )
Response: AGREE
Removing agent 85
Testing agent 33 (comment=Person who believes the following 1 things: (1) Reactionary liberal trash

In [49]:
committee_supporters

{'Raising the minimum wage is a crucial step in addressing income inequality and ensuring employees receive a livable wage, but the increase should be gradual and consider the potential impacts on small businesses and the adoption of automation in industries with unskilled workers.': [215,
  85,
  14,
  55,
  92,
  56,
  52,
  63,
  6,
  47,
  12,
  28,
  15,
  96,
  5989],
 'Our current social welfare programs are failing to address the systemic issues that give rise to the need for these supports in the first place.': [215,
  85,
  33,
  55,
  92,
  56,
  52,
  47,
  110,
  28,
  15,
  96,
  5989]}

### Example EJR slate 

In [30]:
query1_prime_moderator = Query1PrimeModerator(id=0, prompt_type="basic_more_specific")

In [31]:
committee_supporters_ejr = generate_ejr_slate(
    algorithm_type="EJR",
    agents=agents,
    query1_moderator=moderator,
    query1_prime_moderator=query1_prime_moderator,
    k=5,
    experiment_type="polis",
    log_dir_name="demos",
)

Running EJR...
Query 1 response: Addressing income inequality is an important issue, and finding a balance between a livable wage and the effects on small businesses and automation should be considered in any solution.
Testing whether agent 215 (comment=Person who believes the following 1 things: (1) Businesses, small or large, shouldn't be able to pay workers anything below a livable wage. Why should we subsidize businesses that depend on inadequate wages? If a business's product is good, then this shouldn't be a challenge. ) supports new addition
Testing whether agent 85 (comment=Person who believes the following 1 things: (1) It's just going to speed up the adoption of robotics in industries with unskilled or low-skilled workers. ) supports new addition
Testing whether agent 33 (comment=Person who believes the following 1 things: (1) Reactionary liberal trash. Doesn't solve the problem. It's a systemic problem and this solution is like a band aid on a broken leg. ) supports new addi

In [32]:
committee_supporters_ejr

{'Addressing income inequality is important, and finding a fair and effective solution to ensure workers earn a livable wage, while considering the potential impacts on small businesses and technology adoption, should be a priority.': [215,
  85,
  14,
  55,
  92,
  56,
  52,
  63,
  6,
  47,
  12,
  110,
  28,
  15,
  5989],
 'Efforts to address income inequality should consider both the opportunities and risks associated with higher minimum wages, including the potential effects on small businesses, technology adoption, and overall economic well-being, in order to strike a balance that benefits the majority.': [215,
  85,
  14,
  55,
  92,
  56,
  52,
  63,
  6,
  47,
  12,
  110,
  28,
  15,
  96,
  5989],
 'Addressing income inequality requires careful evaluation of potential solutions, such as raising the minimum wage, while considering the effects on small businesses, job opportunities and technological advancements in order to promote economic stability and fairness.': [215,
  8